In [ ]:
"""
NFL Big Data Bowl 2026 - INFERENCE ONLY
使用已训练好的权重进行预测
"""

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime
import warnings
import os
import pickle
import polars as pl
import kaggle_evaluation.nfl_inference_server

# ============================================================================
# CONFIG
# ============================================================================

class Config:
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    WINDOW_SIZE = 8
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)

# ============================================================================
# FEATURE ENGINEERING (与训练时保持一致)
# ============================================================================

def height_to_feet(height_str):
    try:
        ft, inches = map(int, str(height_str).split('-'))
        return ft + inches/12
    except:
        return 6.0

def prepare_sequences_fixed(input_df, output_df=None, test_template=None, is_training=True, window_size=8):
    """
    与训练时完全相同的特征工程
    """
    print(f"Preparing sequences (window_size={window_size})...")
    
    input_df = input_df.copy()
    
    # Basic features
    input_df['player_height_feet'] = input_df['player_height'].apply(height_to_feet)
    
    # Velocity
    dir_rad = np.deg2rad(input_df['dir'].fillna(0))
    delta_t = 0.1
    input_df['velocity_x'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.sin(dir_rad)
    input_df['velocity_y'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.cos(dir_rad)
    
    # Acceleration
    input_df['acceleration_x'] = input_df['a'] * np.sin(dir_rad)
    input_df['acceleration_y'] = input_df['a'] * np.cos(dir_rad)
    
    # Roles
    input_df['is_offense'] = (input_df['player_side'] == 'Offense').astype(int)
    input_df['is_defense'] = (input_df['player_side'] == 'Defense').astype(int)
    input_df['is_receiver'] = (input_df['player_role'] == 'Targeted Receiver').astype(int)
    input_df['is_coverage'] = (input_df['player_role'] == 'Defensive Coverage').astype(int)
    input_df['is_passer'] = (input_df['player_role'] == 'Passer').astype(int)
    
    # Physics
    mass_kg = input_df['player_weight'].fillna(200.0) / 2.20462
    input_df['momentum_x'] = input_df['velocity_x'] * mass_kg
    input_df['momentum_y'] = input_df['velocity_y'] * mass_kg
    input_df['kinetic_energy'] = 0.5 * mass_kg * (input_df['s'] ** 2)
    
    # Ball features
    if 'ball_land_x' in input_df.columns:
        ball_dx = input_df['ball_land_x'] - input_df['x']
        ball_dy = input_df['ball_land_y'] - input_df['y']
        input_df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
        input_df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
        input_df['ball_direction_x'] = ball_dx / (input_df['distance_to_ball'] + 1e-6)
        input_df['ball_direction_y'] = ball_dy / (input_df['distance_to_ball'] + 1e-6)
        input_df['closing_speed'] = (
            input_df['velocity_x'] * input_df['ball_direction_x'] +
            input_df['velocity_y'] * input_df['ball_direction_y']
        )
    
    # Sort for temporal features
    input_df = input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    # Lag features
    for lag in [1, 2, 3]:
        input_df[f'x_lag{lag}'] = input_df.groupby(gcols)['x'].shift(lag)
        input_df[f'y_lag{lag}'] = input_df.groupby(gcols)['y'].shift(lag)
        input_df[f'velocity_x_lag{lag}'] = input_df.groupby(gcols)['velocity_x'].shift(lag)
        input_df[f'velocity_y_lag{lag}'] = input_df.groupby(gcols)['velocity_y'].shift(lag)
    
    # EMA features
    input_df['velocity_x_ema'] = input_df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['velocity_y_ema'] = input_df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['speed_ema'] = input_df.groupby(gcols)['s'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    
    # Rolling features
    input_df['velocity_x_roll'] = input_df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.rolling(window_size, min_periods=1).mean()
    )
    input_df['velocity_y_roll'] = input_df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.rolling(window_size, min_periods=1).mean()
    )
    
    # Feature list
    feature_cols = [
        'x', 'y', 's', 'a', 'o', 'dir', 'frame_id',
        'ball_land_x', 'ball_land_y',
        'player_height_feet', 'player_weight',
        'velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y',
        'momentum_x', 'momentum_y', 'kinetic_energy',
        'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer',
        'distance_to_ball', 'angle_to_ball', 'ball_direction_x', 'ball_direction_y', 'closing_speed',
        'x_lag1', 'y_lag1', 'velocity_x_lag1', 'velocity_y_lag1',
        'x_lag2', 'y_lag2', 'velocity_x_lag2', 'velocity_y_lag2',
        'x_lag3', 'y_lag3', 'velocity_x_lag3', 'velocity_y_lag3',
        'velocity_x_ema', 'velocity_y_ema', 'speed_ema',
        'velocity_x_roll', 'velocity_y_roll',
    ]
    
    feature_cols = [c for c in feature_cols if c in input_df.columns]
    print(f"Using {len(feature_cols)} features")
    
    # Create sequences
    input_df.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
    grouped = input_df.groupby(level=['game_id', 'play_id', 'nfl_id'])
    
    target_rows = output_df if is_training else test_template
    target_groups = target_rows[['game_id', 'play_id', 'nfl_id']].drop_duplicates()
    
    sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = [], [], [], [], []
    
    for _, row in tqdm(target_groups.iterrows(), total=len(target_groups)):
        key = (row['game_id'], row['play_id'], row['nfl_id'])
        
        try:
            group_df = grouped.get_group(key)
        except KeyError:
            continue
        
        input_window = group_df.tail(window_size)
        
        if len(input_window) < window_size:
            if is_training:
                continue
            pad_len = window_size - len(input_window)
            pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
            input_window = pd.concat([pad_df, input_window], ignore_index=True)
        
        input_window = input_window.fillna(group_df.mean(numeric_only=True))
        seq = input_window[feature_cols].values
        
        if np.isnan(seq).any():
            seq = np.nan_to_num(seq, nan=0.0)
        
        sequences.append(seq)
        
        if is_training:
            out_grp = output_df[
                (output_df['game_id']==row['game_id']) &
                (output_df['play_id']==row['play_id']) &
                (output_df['nfl_id']==row['nfl_id'])
            ].sort_values('frame_id')
            
            last_x = input_window.iloc[-1]['x']
            last_y = input_window.iloc[-1]['y']
            
            dx = out_grp['x'].values - last_x
            dy = out_grp['y'].values - last_y
            
            targets_dx.append(dx)
            targets_dy.append(dy)
            targets_frame_ids.append(out_grp['frame_id'].values)
        
        sequence_ids.append({
            'game_id': key[0],
            'play_id': key[1],
            'nfl_id': key[2],
            'frame_id': input_window.iloc[-1]['frame_id']
        })
    
    print(f"Created {len(sequences)} sequences")
    
    if is_training:
        return sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids
    return sequences, sequence_ids

# ============================================================================
# LOSS (与训练时相同)
# ============================================================================

class TemporalHuber(nn.Module):
    def __init__(self, delta=0.5, time_decay=0.03):
        super().__init__()
        self.delta = delta
        self.time_decay = time_decay
    
    def forward(self, pred, target, mask):
        err = pred - target
        abs_err = torch.abs(err)
        
        huber = torch.where(
            abs_err <= self.delta,
            0.5 * err * err,
            self.delta * (abs_err - 0.5 * self.delta)
        )
        
        if self.time_decay > 0:
            L = pred.size(1)
            t = torch.arange(L, device=pred.device).float()
            weight = torch.exp(-self.time_decay * t).view(1, L)
            huber = huber * weight
            mask = mask * weight
        
        return (huber * mask).sum() / (mask.sum() + 1e-8)

# ============================================================================
# MODEL (与训练时相同)
# ============================================================================

class ImprovedSeqModel(nn.Module):
    def __init__(self, input_dim, horizon):
        super().__init__()
        self.horizon = horizon
        
        self.gru = nn.GRU(input_dim, 128, num_layers=2, batch_first=True, dropout=0.1)
        
        self.pool_ln = nn.LayerNorm(128)
        self.pool_attn = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
        self.pool_query = nn.Parameter(torch.randn(1, 1, 128))
        
        self.head = nn.Sequential(
            nn.Linear(128, 128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, horizon)
        )
    
    def forward(self, x):
        h, _ = self.gru(x)
        
        B = h.size(0)
        q = self.pool_query.expand(B, -1, -1)
        h_norm = self.pool_ln(h)
        ctx, _ = self.pool_attn(q, h_norm, h_norm)
        ctx = ctx.squeeze(1)
        
        out = self.head(ctx)
        out = torch.cumsum(out, dim=1)
        
        return out

# ============================================================================
# INFERENCE ONLY - 加载已有模型和scaler
# ============================================================================

def load_models_once():
    global _models_x, _models_y, _scalers, _feature_cols, _models_loaded
    
    print("Loading pre-trained models and scalers...")
    
    # 模型路径
    model_x_paths = [
        "/kaggle/input/nflllll/kaggle/working/models/model_x_fold1.pth",
        "/kaggle/input/nflllll/kaggle/working/models/model_x_fold2.pth",
        "/kaggle/input/nflllll/kaggle/working/models/model_x_fold3.pth",
        "/kaggle/input/nflllll/kaggle/working/models/model_x_fold4.pth",
        "/kaggle/input/nflllll/kaggle/working/models/model_x_fold5.pth"
    ]
    
    model_y_paths = [
        "/kaggle/input/nflllll/kaggle/working/models/model_y_fold1.pth",
        "/kaggle/input/nflllll/kaggle/working/models/model_y_fold2.pth",
        "/kaggle/input/nflllll/kaggle/working/models/model_y_fold3.pth",
        "/kaggle/input/nflllll/kaggle/working/models/model_y_fold4.pth",
        "/kaggle/input/nflllll/kaggle/working/models/model_y_fold5.pth"
    ]
    
    scaler_paths = [
        "/kaggle/input/nflllll/kaggle/working/models/scaler_fold1.pkl",
        "/kaggle/input/nflllll/kaggle/working/models/scaler_fold2.pkl",
        "/kaggle/input/nflllll/kaggle/working/models/scaler_fold3.pkl",
        "/kaggle/input/nflllll/kaggle/working/models/scaler_fold4.pkl",
        "/kaggle/input/nflllll/kaggle/working/models/scaler_fold5.pkl",
    ]
    
    # 确保路径存在
    for path in model_x_paths + model_y_paths + scaler_paths:
        if not os.path.exists(path):
            # 如果在/kaggle/input/nfllll/不存在，尝试在/kaggle/working/models/
            alt_path = path.replace('/kaggle/input/nfllll/', '/kaggle/working/models/')
            if os.path.exists(alt_path):
                path = alt_path
            else:
                raise FileNotFoundError(f"Model file not found: {path}")
    
    _models_x = []
    _models_y = []
    _scalers = []
    
    for i in range(Config.N_FOLDS):
        # 加载模型X
        model_x = ImprovedSeqModel(input_dim=45, horizon=Config.MAX_FUTURE_HORIZON).to(Config.DEVICE)
        model_x.load_state_dict(torch.load(model_x_paths[i], map_location=Config.DEVICE))
        model_x.eval()
        _models_x.append(model_x)
        
        # 加载模型Y
        model_y = ImprovedSeqModel(input_dim=45, horizon=Config.MAX_FUTURE_HORIZON).to(Config.DEVICE)
        model_y.load_state_dict(torch.load(model_y_paths[i], map_location=Config.DEVICE))
        model_y.eval()
        _models_y.append(model_y)
        
        # 加载scaler
        with open(scaler_paths[i], 'rb') as f:
            scaler = pickle.load(f)
        _scalers.append(scaler)
    
    _models_loaded = True
    print("✅ All models and scalers loaded successfully!")

_models_x = []
_models_y = []
_scalers = []
_models_loaded = False
_feature_cols = None

load_models_once()
# ============================================================================
# PREDICT FUNCTION
# ============================================================================

def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    global _models_x, _models_y, _scalers, _models_loaded, _feature_cols
    
    # 第一次调用时加载模型
    if not _models_loaded:
        load_models_once()
    
    test_pd = test.to_pandas()
    test_input_pd = test_input.to_pandas()
    
    print("\nPreparing test sequences...")
    sequences, sequence_ids = prepare_sequences_fixed(
        test_input_pd, test_template=test_pd, is_training=False, window_size=Config.WINDOW_SIZE
    )
    
    X_test = np.array(sequences, dtype=object)
    
    # 获取最后一帧的位置作为基准
    x_last = np.array([s[-1, 0] for s in X_test])
    y_last = np.array([s[-1, 1] for s in X_test])
    
    # 预测
    all_dx, all_dy = [], []
    
    for mx, my, sc in zip(_models_x, _models_y, _scalers):
        # 特征缩放
        X_scaled = np.stack([sc.transform(s) for s in X_test])
        X_tensor = torch.tensor(X_scaled.astype(np.float32)).to(Config.DEVICE)
        
        # 预测
        mx.eval()
        my.eval()
        
        with torch.no_grad():
            dx = mx(X_tensor).cpu().numpy()
            dy = my(X_tensor).cpu().numpy()
        
        all_dx.append(dx)
        all_dy.append(dy)
    
    # 平均预测结果
    ens_dx = np.mean(all_dx, axis=0)
    ens_dy = np.mean(all_dy, axis=0)
    
    # 创建提交结果
    rows = []
    H = ens_dx.shape[1]
    
    for i, sid in enumerate(sequence_ids):
        # 获取该球员在测试集中的所有frame_id
        fids = test_template[
            (test_template['game_id'] == sid['game_id']) &
            (test_template['play_id'] == sid['play_id']) &
            (test_template['nfl_id'] == sid['nfl_id'])
        ]['frame_id'].sort_values().tolist()
        
        for t, fid in enumerate(fids):
            tt = min(t, H - 1)  # 确保不越界
            px = np.clip(x_last[i] + ens_dx[i, tt], 0, 120)
            py = np.clip(y_last[i] + ens_dy[i, tt], 0, 53.3)
            
            rows.append({
                'x': px,
                'y': py
            })
    
    predictions = pl.DataFrame(rows)
    
    assert isinstance(predictions, (pd.DataFrame, pl.DataFrame))
    assert len(predictions) == len(test)  # 确保预测数量与输入一致
    return predictions

# 加载测试模板（用于获取frame_id）
test_template = pd.read_csv(Config.DATA_DIR / "test.csv")

# 设置推理服务器
inference_server = kaggle_evaluation.nfl_inference_server.NFLInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway()